# Configuration tutorial

The Config class prints the same content of the yaml file

In [1]:
from config_util import Config, instantiate

cfg = Config("example_configs/config_base.yaml")
print(cfg)

experiment_name: baseline_experiment
seed: 42
model:
  _target_: Linear
  in_features: 128
  out_features: 10
  bias: true
optimizer:
  _target_: Adam
  lr: 0.001
  weight_decay: 0.0
data:
  batch_size: 32
  num_workers: 4
  path: ./data/dogs
  augmentations:
  - RandomCrop
  - RandomHorizontalFlip



You can access elements with dot notation or use the object as a dictionary

In [2]:
print(cfg.model.in_features)
print(cfg.model["in_features"])
print("in_features" in cfg.model)

128
128
True


Other yaml files can be used for changing or adding new configurations

In [3]:
# config_override changes the experiment name and the optimizer
cfg.update_from_yaml("example_configs/config_override.yaml", allow_extra=True) # Allow new keys
print(cfg)

experiment_name: finetuning_experiment
seed: 42
model:
  _target_: Linear
  in_features: 128
  out_features: 10
  bias: true
optimizer:
  _target_: SGD
  lr: 0.01
  weight_decay: 0.0
  momentum: 0.9
data:
  batch_size: 32
  num_workers: 4
  path: ./data/dogs
  augmentations:
  - RandomCrop
  - RandomHorizontalFlip



The configuration can also be changed from a dictionary nested using the same structure as the yaml file

In [4]:
overrides = {
    "model": {
        "out_features": 2  # Change to 2 classes instead of 10
    },
    "data": {
        "batch_size": 64  # Change batch size
    }
}

cfg.update_from_dict(overrides) 
print(cfg)

experiment_name: finetuning_experiment
seed: 42
model:
  _target_: Linear
  in_features: 128
  out_features: 2
  bias: true
optimizer:
  _target_: SGD
  lr: 0.01
  weight_decay: 0.0
  momentum: 0.9
data:
  batch_size: 64
  num_workers: 4
  path: ./data/dogs
  augmentations:
  - RandomCrop
  - RandomHorizontalFlip



update_from_* can add new keys and update current ones. If you want to completely replace some configuration, just assing to it

In [5]:
cfg.data = Config("example_configs/config_data2.yaml").data
print(cfg)

experiment_name: finetuning_experiment
seed: 42
model:
  _target_: Linear
  in_features: 128
  out_features: 2
  bias: true
optimizer:
  _target_: SGD
  lr: 0.01
  weight_decay: 0.0
  momentum: 0.9
data:
  batch_size: 32
  path: ./data/cats



The final configuration can be saved to a yaml file

In [6]:
cfg.save("example_configs/config_final.yaml")

The class indicated in _target_ can be **instantiated** with all parameters on the same level (unless they begin with _)

In [7]:
model = instantiate(cfg.model)
model

Linear(in_features=128, out_features=2, bias=True)

Lazy instantiation is also supported in case the constructor needs additional objects created at runtime

In [8]:
optimizer_factory = instantiate(cfg.optimizer, partial=True)
optimizer = optimizer_factory(model.parameters())
optimizer


SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0
)